In [3]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import numpy as np

# Create SQLite database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Drop existing tables if they exist
tables = ['order_items', 'orders', 'products', 'customers', 'suppliers']
for table in tables:
    cursor.execute(f'DROP TABLE IF EXISTS {table}')

# Create Customers table
cursor.execute('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE,
    phone TEXT,
    registration_date DATE,
    city TEXT,
    age INTEGER
)
''')

# Create Products table
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price DECIMAL(10,2),
    stock_quantity INTEGER,
    supplier_id INTEGER,
    rating DECIMAL(3,2)
)
''')

# Create Orders table
cursor.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount DECIMAL(10,2),
    status TEXT,
    shipping_cost DECIMAL(6,2),
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
)
''')

# Create Order_Items table
cursor.execute('''
CREATE TABLE order_items (
    item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    unit_price DECIMAL(10,2),
    discount DECIMAL(5,2),
    FOREIGN KEY (order_id) REFERENCES orders (order_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
)
''')

# Create Suppliers table
cursor.execute('''
CREATE TABLE suppliers (
    supplier_id INTEGER PRIMARY KEY,
    supplier_name TEXT NOT NULL,
    contact_person TEXT,
    email TEXT,
    phone TEXT,
    address TEXT,
    country TEXT,
    performance_score DECIMAL(3,2)
)
''')

# Insert expanded sample data into Customers
customers_data = [
    (1, 'John', 'Doe', 'john.doe@email.com', '555-1234', '2023-01-15', 'New York', 28),
    (2, 'Jane', 'Smith', 'jane.smith@email.com', '555-5678', '2023-02-20', 'Los Angeles', 34),
    (3, 'Mike', 'Johnson', 'mike.j@email.com', '555-9012', '2023-03-10', 'Chicago', 45),
    (4, 'Sarah', 'Williams', 'sarah.w@email.com', '555-3456', '2023-04-05', 'Houston', 29),
    (5, 'David', 'Brown', 'david.brown@email.com', '555-7890', '2023-05-12', 'Phoenix', 38),
    (6, 'Emma', 'Davis', 'emma.davis@email.com', '555-4567', '2023-06-18', 'Philadelphia', 31),
    (7, 'Chris', 'Wilson', 'chris.w@email.com', '555-8901', '2023-07-22', 'San Antonio', 42),
    (8, 'Lisa', 'Miller', 'lisa.miller@email.com', '555-2345', '2023-08-15', 'San Diego', 26),
    (9, 'Robert', 'Taylor', 'robert.t@email.com', '555-6789', '2023-09-08', 'Dallas', 39),
    (10, 'Maria', 'Garcia', 'maria.garcia@email.com', '555-0123', '2023-10-12', 'San Jose', 33)
]
cursor.executemany('INSERT INTO customers VALUES (?,?,?,?,?,?,?,?)', customers_data)

# Insert sample data into Suppliers
suppliers_data = [
    (1, 'TechCorp', 'Alice Manager', 'alice@techcorp.com', '555-1111', '123 Tech St', 'USA', 4.5),
    (2, 'ElectroSupply', 'Bob Sales', 'bob@electro.com', '555-2222', '456 Electric Ave', 'USA', 4.2),
    (3, 'FoodDistrib', 'Carol Admin', 'carol@fooddist.com', '555-3333', '789 Food Blvd', 'USA', 4.0),
    (4, 'GlobalTech', 'David Tech', 'david@globaltech.com', '555-4444', '321 Global Way', 'Canada', 4.7),
    (5, 'HomeGoods', 'Eva Home', 'eva@homegoods.com', '555-5555', '654 Home Street', 'USA', 4.3)
]
cursor.executemany('INSERT INTO suppliers VALUES (?,?,?,?,?,?,?,?)', suppliers_data)

# Insert expanded sample data into Products
products_data = [
    (1, 'Laptop', 'Electronics', 999.99, 50, 1, 4.5),
    (2, 'Smartphone', 'Electronics', 699.99, 100, 1, 4.3),
    (3, 'Headphones', 'Electronics', 199.99, 75, 2, 4.1),
    (4, 'Coffee Maker', 'Appliances', 149.99, 30, 2, 4.0),
    (5, 'Organic Coffee', 'Food', 24.99, 200, 3, 4.4),
    (6, 'Wireless Mouse', 'Electronics', 49.99, 120, 1, 4.2),
    (7, 'Protein Bars', 'Food', 19.99, 150, 3, 3.9),
    (8, 'Tablet', 'Electronics', 499.99, 60, 4, 4.6),
    (9, 'Smart Watch', 'Electronics', 299.99, 80, 4, 4.4),
    (10, 'Blender', 'Appliances', 89.99, 40, 5, 4.1),
    (11, 'Yoga Mat', 'Sports', 39.99, 90, 5, 4.3),
    (12, 'Gaming Chair', 'Furniture', 249.99, 25, 2, 4.2)
]
cursor.executemany('INSERT INTO products VALUES (?,?,?,?,?,?,?)', products_data)

# Insert expanded sample data into Orders
orders_data = [
    (1, 1, '2023-06-01', 1249.98, 'Completed', 9.99),
    (2, 2, '2023-06-02', 719.98, 'Completed', 7.99),
    (3, 3, '2023-06-03', 199.99, 'Shipped', 5.99),
    (4, 4, '2023-06-04', 174.98, 'Processing', 6.99),
    (5, 5, '2023-06-05', 69.98, 'Completed', 4.99),
    (6, 6, '2023-06-10', 799.98, 'Completed', 8.99),
    (7, 7, '2023-06-15', 349.98, 'Shipped', 6.99),
    (8, 8, '2023-06-20', 129.98, 'Completed', 5.99),
    (9, 9, '2023-06-25', 589.97, 'Processing', 7.99),
    (10, 10, '2023-06-30', 89.99, 'Completed', 4.99),
    (11, 1, '2023-07-05', 299.99, 'Completed', 6.99),
    (12, 2, '2023-07-10', 449.98, 'Shipped', 7.99)
]
cursor.executemany('INSERT INTO orders VALUES (?,?,?,?,?,?)', orders_data)

# Insert expanded sample data into Order_Items
order_items_data = [
    (1, 1, 1, 1, 999.99, 0.0),   # Order 1: 1 Laptop
    (2, 1, 6, 5, 49.99, 5.0),    # Order 1: 5 Wireless Mouse with 5% discount
    (3, 2, 2, 1, 699.99, 0.0),   # Order 2: 1 Smartphone
    (4, 2, 5, 1, 24.99, 0.0),    # Order 2: 1 Organic Coffee
    (5, 3, 3, 1, 199.99, 0.0),   # Order 3: 1 Headphones
    (6, 4, 4, 1, 149.99, 0.0),   # Order 4: 1 Coffee Maker
    (7, 4, 5, 1, 24.99, 0.0),    # Order 4: 1 Organic Coffee
    (8, 5, 6, 1, 49.99, 0.0),    # Order 5: 1 Wireless Mouse
    (9, 5, 7, 1, 19.99, 0.0),    # Order 5: 1 Protein Bars
    (10, 6, 8, 1, 499.99, 10.0), # Order 6: 1 Tablet with 10% discount
    (11, 6, 9, 1, 299.99, 0.0),  # Order 6: 1 Smart Watch
    (12, 7, 9, 1, 299.99, 0.0),  # Order 7: 1 Smart Watch
    (13, 7, 6, 1, 49.99, 0.0),   # Order 7: 1 Wireless Mouse
    (14, 8, 10, 1, 89.99, 0.0),  # Order 8: 1 Blender
    (15, 8, 11, 1, 39.99, 0.0),  # Order 8: 1 Yoga Mat
    (16, 9, 12, 1, 249.99, 0.0), # Order 9: 1 Gaming Chair
    (17, 9, 9, 1, 299.99, 15.0), # Order 9: 1 Smart Watch with 15% discount
    (18, 9, 11, 1, 39.99, 0.0),  # Order 9: 1 Yoga Mat
    (19, 10, 10, 1, 89.99, 0.0), # Order 10: 1 Blender
    (20, 11, 9, 1, 299.99, 0.0), # Order 11: 1 Smart Watch
    (21, 12, 8, 1, 499.99, 10.0),# Order 12: 1 Tablet with 10% discount
    (22, 12, 3, 1, 199.99, 0.0)  # Order 12: 1 Headphones
]
cursor.executemany('INSERT INTO order_items VALUES (?,?,?,?,?,?)', order_items_data)

# Commit changes
conn.commit()

print("Enhanced SQLite database 'text2sql_demo.db' created successfully!")
print("Database now includes:")
print("- 5 tables with foreign key relationships")
print("- 10 customers with demographics")
print("- 12 products across multiple categories")
print("- 12 orders with various statuses")
print("- 22 order items with discounts")
print("- 5 suppliers with performance scores")
print("\nReady for text2sql queries and data visualization!")

# Close connection
conn.close()

Enhanced SQLite database 'text2sql_demo.db' created successfully!
Database now includes:
- 5 tables with foreign key relationships
- 10 customers with demographics
- 12 products across multiple categories
- 12 orders with various statuses
- 22 order items with discounts
- 5 suppliers with performance scores

Ready for text2sql queries and data visualization!
